Data Cleaning - Aggregated Airbnb Reviews

# Introduction

In the following notebook, I will be cleaning an aggregation of Airbnb Reviews data from different users about their stay in the San Francisco area. This aggregation consists of reviews data from 12/2018 through 12/2019.

The aggregation source code can be found [here](https://github.com/KishenSharma6/Airbnb-Analysis/blob/master/Project%20Codes/01.%20Raw%20Data%20Aggregation%20Scripts/2020_0129_Airbnb_Raw_Data_Aggregation.ipynb)

Raw data can be found [here](http://insideairbnb.com/get-the-data.html)

## Read in libraries, data, and set notebook preferences

**Read in libraries**

In [2]:
#Read in libraries
import pandas as pd
import numpy as np

import swifter

**Read in Data**

In [3]:
#Set path to get aggregated Calendar data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\01_Raw\SF Airbnb Raw Data\SF Airbnb Raw Data - Aggregated\01_04_2020_Reviews_Raw_Aggregated.csv'

#Read in Airbnb Review Data
reviews = pd.read_csv(path, sep=',', parse_dates=['date'],
                      dtype = {'id':'object','listing_id':'object','reviewer_id':'object'},
                      index_col=0)              

**Set Notebook preferences**

In [4]:
#Set Pandas options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',100)

#Ignore warnings
import warnings; warnings.simplefilter('ignore')

## Data Preview

In [5]:
#View data shape and view head
print('Reviews data shape:', reviews.shape)
display(reviews.head())

,comments,date,id,listing_id,reviewer_id,reviewer_name
0,"Our experience was, without a doubt, a five st...",2009-07-23,5977,958,15695,Edmund C
1,Returning to San Francisco is a rejuvenating t...,2009-08-03,6660,958,26145,Simon
2,We were very pleased with the accommodations a...,2009-09-27,11519,958,25839,Denis
3,We highly recommend this accomodation and agre...,2009-11-05,16282,958,33750,Anna
4,Holly's place was great. It was exactly what I...,2010-02-13,26008,958,15416,Venetia


In [6]:
#Show data types
reviews.dtypes

comments                 object
date             datetime64[ns]
id                        int64
listing_id                int64
reviewer_id               int64
reviewer_name            object
dtype: object

# Data Cleaning

## Missing data

**Check for and remove missing data**

In [9]:
#Replace blank comments with NAN
reviews.comments.replace('^\s*$', np.nan, regex=True, inplace=True)

#View missing values
print('Missing values: \n', reviews.isna().sum())

Missing values: 
 comments         480
date               0
id                 0
listing_id         0
reviewer_id        0
reviewer_name      1
dtype: int64


In [10]:
#Remove rows with NA in comments
reviews  = reviews[~reviews.comments.isna()]

#Replace missing reviewer name with Unknown
reviews.reviewer_name.fillna('Unknown', inplace = True)

#Check
print('Missing values: \n', reviews.isna().sum())

Missing values: 
 comments         0
date             0
id               0
listing_id       0
reviewer_id      0
reviewer_name    0
dtype: int64


## Language Detection

Not all comments are in English. We will assign comment languages to a 'language' column and perform a separate NLP for different languages to see if we can extract additional meaning from users about their Airbnb experience in San Francisco

In [ ]:
#Import language detection library
from langdetect import detect

# write the function that detects the language
def language_detection(text):
    try:
        return detect(text)
    except:
        return None

#Apply function to reviews and check
reviews['language']=reviews.comments.swifter.apply(language_detection)
display(reviews.head())

## Comments cleaning

In [ ]:
#Remove \n,\r and \t
reviews.comments.replace('(\\n|\\t|\\r)', ' ',regex=True, inplace=True)

#Replace new blank comments with NAN
reviews.comments.replace('^\s*$', np.nan, regex=True, inplace=True)

#Remove new rows with NA in comments
reviews  = reviews[~reviews.comments.isna()]

#Strip trailing and leading whitespace
reviews['comments'].str.strip()

#Remove rows where comments character string < 3
reviews = reviews[reviews.comments.apply(len) > 3].sort_values(by='comments')

In [39]:
#Print current reviews shape
print('Current shape of reviews:', reviews.shape)

Current shape of reviews: (425548, 6)


In [40]:
#Set path to write listings
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\02_Intermediate\2020_0131_Reviews_Cleaned.csv'

#Write listings to path
reviews.to_csv(path, sep=',')